In [12]:
import pandas as pd
from sko.PSO import PSO
import numpy as np
import matplotlib.pyplot as plt
from geopy.distance import distance

In [8]:
fire_data=pd.read_csv("../data/output_data/output_fire_data/clustering_fire_data.csv")
fire_data

,latitude,longitude,frp
0,-36.979083,143.869362,9.0278
1,-37.395876,148.702175,61.1422
2,-36.360385,141.245716,10.0394
3,-37.326302,146.998582,17.1700
4,-37.521991,142.326735,13.3154
...,...,...,...
95,-35.033808,143.126835,1.2592
96,-38.101039,141.752253,6.5914
97,-34.821044,142.132761,3.4122
98,-38.242735,146.371048,21.8620


In [ ]:
def obj_func(x_cors,y_cors):
    res=0
    for i in range(len(fire_data)):
        res+=fire_data.loc[i,['frp']]
        for j in range(len(x_cors)):
            if(distance((x_cors[j],y_cors[j]),(fire_data.loc[i,['longitude']],fire_data.loc[i,['latitude']])).km>=5):
                res=res-fire_data.loc[i,['frp']]
                break

In [ ]:
drone_num=30
pso=PSO(func=obj_func,)